In [ ]:

import numpy as np
import math
import scipy.io as sio
from sklearn.model_selection import train_test_split

# Structure of the 3-layer neural network.
Input_layer_size = 400
Hidden_layer_size = 25
Output_layer_size = 10


def convert_memory_ordering_f2c(array):
    if np.isfortran(array) is True:
        return np.ascontiguousarray(array)
    else:
        return array


def load_training_data(training_file='mnistdata.mat'):
    '''Load training data (mnistdata.mat) and return (inputs, labels).

    inputs: numpy array with size (5000, 400).
    labels: numpy array with size (5000, 1).

    The training data is from Andrew Ng's exercise of the Coursera
    machine learning course (ex4data1.mat).
    '''
    training_data = sio.loadmat(training_file)
    inputs = training_data['X'].astype('f8')
    inputs = convert_memory_ordering_f2c(inputs)
    labels = training_data['y']
    labels = convert_memory_ordering_f2c(labels)
    return (inputs, labels)


def load_weights(weight_file='mnistweights.mat'):
    '''Load training data (mnistdata.mat) and return (inputs, labels).

    The weights file is from Andrew Ng's exercise of the Coursera
    machine learning course (ex4weights.mat).
    '''
    weights = sio.loadmat(weight_file)
    theta1 = convert_memory_ordering_f2c(weights['Theta1'].astype('f8'))  # size: 25 entries, each has 401 numbers
    theta2 = convert_memory_ordering_f2c(weights['Theta2'].astype('f8'))  # size: 10 entries, each has  26 numbers
    return (theta1, theta2)


def rand_init_weights(size_in, size_out):
    epsilon_init = 0.12
    return np.random.rand(size_out, 1 + size_in) * 2 * epsilon_init - epsilon_init


def sigmoid(z):
    return 1.0 / (1 + pow(math.e, -z))


def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))


def cost_function(theta1, theta2, input_layer_size, hidden_layer_size, output_layer_size, inputs, labels, regular=0):
    '''
    Note: theta1, theta2, inputs, labels are numpy arrays:

        theta1: (25, 401)
        theta2: (10, 26)
        inputs: (5000, 400)
        labels: (5000, 1)
    '''
    # construct neural network
    input_layer = np.insert(inputs, 0, 1, axis=1)  # add bias, 5000x401

    hidden_layer = np.dot(input_layer, np.transpose(theta1))
    hidden_layer = sigmoid(hidden_layer)
    hidden_layer = np.insert(hidden_layer, 0, 1, axis=1)  # add bias, 5000x26

    output_layer = np.dot(hidden_layer, np.transpose(theta2))  # 5000x10
    output_layer = sigmoid(output_layer)
    #print('input  layer shape: {}'.format(input_layer.shape))
    #print('hidden layer shape: {}'.format(hidden_layer.shape))
    #print('output layer shape: {}'.format(output_layer.shape))

    # forward propagation: calculate cost
    cost = 0.0
    for training_index in range(len(inputs)):
        # transform label y[i] from a number to a vector.
        #
        # Note:
        #   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #    1  2  3  4  5  6  7  8  9 10
        #
        #   if y[i] is 0 -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
        #   if y[i] is 1 -> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        outputs = [0] * output_layer_size
        outputs[int(labels[training_index])-1] = 1

        for k in range(output_layer_size):
            cost += -outputs[k] * math.log(output_layer[training_index][k]) - (1 - outputs[k]) * math.log(1 - output_layer[training_index][k])
    cost /= len(inputs)

    # back propagation: calculate gradiants
    theta1_grad = np.zeros_like(theta1)  # 25x401
    theta2_grad = np.zeros_like(theta2)  # 10x26
    for index in range(len(inputs)):
        # transform label y[i] from a number to a vector.
        outputs = np.zeros((1, output_layer_size))  # (1,10)
        outputs[0][labels[index]-1] = 1

        # calculate delta3
        delta3 = (output_layer[index] - outputs).T  # (10,1)

        # calculate delta2
        z2 = np.dot(theta1, input_layer[index:index+1].T)  # (25,401) x (401,1)
        z2 = np.insert(z2, 0, 1, axis=0)  # add bias, (26,1)
        delta2 = np.multiply(
            np.dot(theta2.T, delta3),  # (26,10) x (10,1)
            sigmoid_gradient(z2)       # (26,1)
        )
        delta2 = delta2[1:]  # (25,1)

        # calculate gradients of theta1 and theta2
        # (25,401) = (25,1) x (1,401)
        theta1_grad += np.dot(delta2, input_layer[index:index+1])
        # (10,26) = (10,1) x (1,26)
        theta2_grad += np.dot(delta3, hidden_layer[index:index+1])
    theta1_grad /= len(inputs)
    theta2_grad /= len(inputs)

    return cost, (theta1_grad, theta2_grad)


def gradient_descent(inputs, labels, learningrate=0.8, iteration=50):
    '''
    @return cost and trained model (weights).
    '''
    rand_theta1 = rand_init_weights(Input_layer_size, Hidden_layer_size)
    rand_theta2 = rand_init_weights(Hidden_layer_size, Output_layer_size)
    theta1 = rand_theta1
    theta2 = rand_theta2
    cost = 0.0
    for i in range(iteration):
        cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2,
            Input_layer_size, Hidden_layer_size, Output_layer_size,
            inputs, labels, regular=0)
        theta1 -= learningrate * theta1_grad
        theta2 -= learningrate * theta2_grad
        print('Iteration {0} (learning rate {2}, iteration {3}), cost: {1}'.format(i+1, cost, learningrate, iteration))
    return cost, (theta1, theta2)


def train(inputs, labels, learningrate=0.8, iteration=50):
    cost, model = gradient_descent(inputs, labels, learningrate, iteration)
    return model


def predict(model, inputs):
    theta1, theta2 = model
    a1 = np.insert(inputs, 0, 1, axis=1)  # add bias, (5000,401)
    a2 = np.dot(a1, theta1.T)  # (5000,401) x (401,25)
    a2 = sigmoid(a2)
    a2 = np.insert(a2, 0, 1, axis=1)  # add bias, (5000,26)
    a3 = np.dot(a2, theta2.T)  # (5000,26) x (26,10)
    a3 = sigmoid(a3)  # (5000,10)
    return [i.argmax()+1 for i in a3]


if __name__ == '__main__':
    # Note: There are 10 units which present the digits [1-9, 0]
    # (in order) in the output layer.
    inputs, labels = load_training_data()
    X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)
    # model = train(X_train, y_train, learningrate=2, iteration=60)
    
    # Đo thời gian bắt đầu huấn luyện
    start_time = time.time()

    # Huấn luyện mô hình
    model = train(X_train, y_train, learningrate=2, iteration=60)

    # Đo thời gian kết thúc huấn luyện
    end_time = time.time()

    # Tính thời gian huấn luyện
    training_time = end_time - start_time
    print(f"Thời gian huấn luyện: {training_time:.2f} giây")

    # train the model from scratch and predict based on it
    # learning rate 0.10, iteration  60: 36% (cost: 3.217)
    # learning rate 1.75, iteration  50: 77%
    # learning rate 1.90, iteration  50: 75%
    # learning rate 2.00, iteration  50: 82%
    # learning rate 2.00, iteration 100: 87%
    # learning rate 2.00, iteration 200: 93% (cost: 0.572)
    # learning rate 2.00, iteration 300: 94% (cost: 0.485)
    # learning rate 2.05, iteration  50: 79%
    # learning rate 2.20, iteration  50: 64%
    # model = train(inputs, labels, learningrate=2, iteration=60)

    # Load pretrained weights for debugging precision.
    # The precision will be around 97% (0.9756).
    #weights = load_weights()
    #theta1 = weights[0]  # size: 25 entries, each has 401 numbers
    #theta2 = weights[1]  # size: 10 entries, each has  26 numbersf
    #model = (theta1, theta2)
    #cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2, Input_layer_size, Hidden_layer_size, Output_layer_size, inputs, labels, regular=0)
    #print('cost:', cost)

    # outputs = predict(model, inputs)
    outputs = predict(model, X_test)
    correct_prediction = sum(outputs[i] == y_test[i][0] for i in range(len(y_test)))
    precision = correct_prediction / len(y_test)

    print('Test precision: {}'.format(precision))

    # correct_prediction = 0
    # for i, predict in enumerate(outputs):
    #     if predict == labels[i][0]:
    #         correct_prediction += 1
    # precision = float(correct_prediction) / len(labels)
    # print('precision: {}'.format(precision))


C:\Users\dotha\AppData\Local\Temp\ipykernel_28560\3699265438.py:125: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outputs[int(labels[training_index])-1] = 1


Iteration 1 (learning rate 2, iteration 60), cost: 7.09596155240651
Iteration 2 (learning rate 2, iteration 60), cost: 5.7448484814036584
Iteration 3 (learning rate 2, iteration 60), cost: 4.11453410280061
Iteration 4 (learning rate 2, iteration 60), cost: 3.28249269655821
Iteration 5 (learning rate 2, iteration 60), cost: 3.264530851478859
Iteration 6 (learning rate 2, iteration 60), cost: 3.2571134638888406
Iteration 7 (learning rate 2, iteration 60), cost: 3.2504783952970935
Iteration 8 (learning rate 2, iteration 60), cost: 3.2438904078036748
Iteration 9 (learning rate 2, iteration 60), cost: 3.2370332338489174
Iteration 10 (learning rate 2, iteration 60), cost: 3.2296931248924285
Iteration 11 (learning rate 2, iteration 60), cost: 3.2216895191584958
Iteration 12 (learning rate 2, iteration 60), cost: 3.212859824079232
Iteration 13 (learning rate 2, iteration 60), cost: 3.203055217563154
Iteration 14 (learning rate 2, iteration 60), cost: 3.1921314192145958
Iteration 15 (learning r

In [ ]:
import numpy as np
import math
import pandas as pd
import scipy.io as sio
# import theano
# import theano.tensor as T
import time
from sklearn.model_selection import train_test_split

# Structure of the 3-layer neural network.
Input_layer_size = 400
Hidden_layer_size = 25
Output_layer_size = 10

# Matrix product function.  Default is to use CPU mode.
Matrix_dot = np.dot


def load_training_data(training_file='mnistdata.mat'):
    '''Load training data (mnistdata.mat) and return (inputs, labels).

    inputs: numpy array with size (5000, 400).
    labels: numpy array with size (5000, 1).

    The training data is from Andrew Ng's exercise of the Coursera
    machine learning course (ex4data1.mat).
    '''
    training_data = sio.loadmat(training_file)
    return (training_data['X'], training_data['y'])


def load_weights(weight_file='mnistweights.mat'):
    '''Load training data (mnistdata.mat) and return (inputs, labels).

    The weights file is from Andrew Ng's exercise of the Coursera
    machine learning course (ex4weights.mat).
    '''
    weights = sio.loadmat(weight_file)
    return weights


def rand_init_weights(size_in, size_out):
    epsilon_init = 0.12
    return np.random.rand(size_out, 1 + size_in) * 2 * epsilon_init - epsilon_init


def sigmoid(z):
    return 1.0 / (1 + pow(math.e, -z))


def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

def gpu_matrix_dot():
    time_start = time.time()
    
    x = np.random.rand(1000, 1000).astype(np.float32)
    y = np.random.rand(1000, 1000).astype(np.float32)
    z = np.dot(x, y)  # Nhân ma trận trong NumPy
    
    def f(x, y):
        return np.dot(x, y)  # Hàm thực hiện nhân ma trận
    
    time_end = time.time()
    print('NumPy function creation costs {} secs'.format(time_end - time_start))
    
    return f



def cost_function(theta1, theta2, input_layer_size, hidden_layer_size, output_layer_size, inputs, labels, regular=0):
    '''
    Note: theta1, theta2, inputs, labels are numpy arrays:

        theta1: (25, 401)
        theta2: (10, 26)
        inputs: (5000, 400)
        labels: (5000, 1)
    '''
    # construct neural network
    input_layer = np.insert(inputs, 0, 1, axis=1)  # add bias, 5000x401

    time_start = time.time()
    hidden_layer = Matrix_dot(input_layer, theta1.T)
    hidden_layer = sigmoid(hidden_layer)
    hidden_layer = np.insert(hidden_layer, 0, 1, axis=1)  # add bias, 5000x26
    time_end = time.time()
    print('\tconstruction: hidden layer dot costs {} secs'.format(time_end - time_start))

    time_start = time.time()
    output_layer = Matrix_dot(hidden_layer, theta2.T)  # 5000x10
    output_layer = sigmoid(output_layer)
    time_end = time.time()
    print('\tconstruction: output layer dot costs {} secs'.format(time_end - time_start))

    # forward propagation: calculate cost
    time_start = time.time()
    cost = 0.0
    for training_index in range(len(inputs)):
        # transform label y[i] from a number to a vector.
        #
        # Note:
        #   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #    1  2  3  4  5  6  7  8  9 10
        #
        #   if y[i] is 0 -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
        #   if y[i] is 1 -> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        outputs = [0] * output_layer_size
        outputs[int(labels[training_index])-1] = 1

        for k in range(output_layer_size):
            cost += -outputs[k] * math.log(output_layer[training_index][k]) - (1 - outputs[k]) * math.log(1 - output_layer[training_index][k])
    cost /= len(inputs)
    time_end = time.time()
    print('\tforward prop: costs {} secs'.format(time_end - time_start))

    # back propagation: calculate gradiants
    time_start = time.time()
    theta1_grad = np.zeros_like(theta1)  # 25x401
    theta2_grad = np.zeros_like(theta2)  # 10x26
    for index in range(len(inputs)):
        # transform label y[i] from a number to a vector.
        outputs = np.zeros((1, output_layer_size))  # (1,10)
        outputs[0][labels[index]-1] = 1

        # calculate delta3
        delta3 = (output_layer[index] - outputs).T  # (10,1)

        # calculate delta2
        z2 = Matrix_dot(theta1, input_layer[index:index+1].T)  # (25,401) x (401,1)
        z2 = np.insert(z2, 0, 1, axis=0)  # add bias, (26,1)
        delta2 = np.multiply(
            Matrix_dot(theta2.T, delta3),  # (26,10) x (10,1)
            sigmoid_gradient(z2)  # (26,1)
        )
        delta2 = delta2[1:]  # (25,1)

        # calculate gradients of theta1 and theta2
        # (25,401) = (25,1) x (1,401)
        theta1_grad += Matrix_dot(delta2, input_layer[index:index+1])
        # (10,26) = (10,1) x (1,26)
        theta2_grad += Matrix_dot(delta3, hidden_layer[index:index+1])
    theta1_grad /= len(inputs)
    theta2_grad /= len(inputs)
    time_end = time.time()
    print('\tback prop: costs {} secs'.format(time_end - time_start))

    return cost, (theta1_grad, theta2_grad)


def gradient_descent(inputs, labels, learningrate=0.8, iteration=50):
    '''
    @return cost and trained model (weights).
    '''
    rand_theta1 = rand_init_weights(Input_layer_size, Hidden_layer_size)
    rand_theta2 = rand_init_weights(Hidden_layer_size, Output_layer_size)
    theta1 = rand_theta1
    theta2 = rand_theta2
    cost = 0.0
    for i in range(iteration):
        time_start = time.time()
        cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2,
            Input_layer_size, Hidden_layer_size, Output_layer_size,
            inputs, labels, regular=0)
        theta1 -= learningrate * theta1_grad
        theta2 -= learningrate * theta2_grad
        time_end = time.time()
        print('Iteration {0} (learning rate {2}, iteration {3}), cost: {1}, time: {4}'.format(i+1, cost, learningrate, iteration, time_end - time_start))
    return cost, (theta1, theta2)


def train(inputs, labels, learningrate=0.8, iteration=50):
    cost, model = gradient_descent(inputs, labels, learningrate, iteration)
    return model


def predict(model, inputs):
    theta1, theta2 = model
    a1 = np.insert(inputs, 0, 1, axis=1)  # add bias, (5000,401)
    a2 = np.dot(a1, theta1.T)  # (5000,401) x (401,25)
    a2 = sigmoid(a2)
    a2 = np.insert(a2, 0, 1, axis=1)  # add bias, (5000,26)
    a3 = np.dot(a2, theta2.T)  # (5000,26) x (26,10)
    a3 = sigmoid(a3)  # (5000,10)
    return [i.argmax()+1 for i in a3]


if __name__ == '__main__':
    gpu_mode = True
    #gpu_mode = False
    if gpu_mode is True:
        print('GPU mode')
        Matrix_dot = gpu_matrix_dot()
    else:
        print('CPU mode')
        Matrix_dot = np.dot

    # Note: There are 10 units which present the digits [1-9, 0]
    # (in order) in the output layer.
    inputs, labels = load_training_data()
    # Chia dữ liệu thành 80% train và 20% test
    X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)


    # (optional) load pre-trained model for debugging neural network construction
    weights = load_weights()
    theta1 = weights['Theta1']  # size: 25 entries, each has 401 numbers
    theta2 = weights['Theta2']  # size: 10 entries, each has  26 numbers

    # FIXME: Memory alignment issue in Scipy.
    #   This issue leads Theano to complain that "The numpy.ndarray
    #   object is not aligned. Theano C code does not support that."
    #
    #   Related discussion: http://stackoverflow.com/questions/36321400/strange-typeerror-with-theano/36323861
    # workaround to avoid memory alignment error in Scipy
    theta1 = np.array(theta1)
    theta2 = np.array(theta2)
    #print('theta1: {}'.format(theta1))
    #print('theta1 flags: {}'.format(theta1.flags))
    #print('theta2: {}'.format(theta2))
    #print('theta2 flags: {}'.format(theta2.flags))
 
    cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2, Input_layer_size, Hidden_layer_size, Output_layer_size, inputs, labels, regular=0)
    print('cost:', cost)

    # train the model from scratch and predict based on it
    # learning rate 0.10, iteration  60: 36% (cost: 3.217)
    # learning rate 1.75, iteration  50: 77%
    # learning rate 1.90, iteration  50: 75%
    # learning rate 2.00, iteration  50: 82%
    # learning rate 2.00, iteration 100: 87%
    # learning rate 2.00, iteration 200: 93% (cost: 0.572)
    # learning rate 2.00, iteration 300: 94% (cost: 0.485)
    # learning rate 2.05, iteration  50: 79%
    # learning rate 2.20, iteration  50: 64%
    # Bắt đầu đo thời gian huấn luyện
    start_time = time.time()

    # Huấn luyện mô hình trên tập train
    model = train(X_train, y_train, learningrate=2, iteration=60)

    # Kết thúc đo thời gian huấn luyện
    end_time = time.time()

    # Tính tổng thời gian huấn luyện
    training_time = end_time - start_time
    print(f"Thời gian huấn luyện: {training_time:.2f} giây")
    
    outputs = predict(model, X_test)
    # model = train(inputs, labels, learningrate=2, iteration=60)
    #model = (theta1, theta2)
    # outputs = predict(model, inputs)

    correct_prediction = sum(outputs[i] == y_test[i][0] for i in range(len(y_test)))
    precision = correct_prediction / len(y_test)

    print(f'Độ chính xác trên tập test: {precision:.4f}')


    # correct_prediction = 0
    # for i, predict in enumerate(outputs):
    #     if predict == labels[i][0]:
    #         correct_prediction += 1
    # precision = float(correct_prediction) / len(labels)
    # print('precision: {}'.format(precision))


GPU mode
NumPy function creation costs 0.025166988372802734 secs
	construction: hidden layer dot costs 0.007000923156738281 secs
	construction: output layer dot costs 0.002003908157348633 secs
	forward prop: costs 0.07242369651794434 secs


C:\Users\dotha\AppData\Local\Temp\ipykernel_28560\1449145469.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outputs[int(labels[training_index])-1] = 1


	back prop: costs 0.9335758686065674 secs
cost: 0.2876291651613203
	construction: hidden layer dot costs 0.005045175552368164 secs
	construction: output layer dot costs 0.0014371871948242188 secs
	forward prop: costs 0.04389476776123047 secs
	back prop: costs 0.533027172088623 secs
Iteration 1 (learning rate 2, iteration 60), cost: 7.086078220576518, time: 0.5880126953125
	construction: hidden layer dot costs 0.003339529037475586 secs
	construction: output layer dot costs 0.0020093917846679688 secs
	forward prop: costs 0.04181408882141113 secs
	back prop: costs 0.5019538402557373 secs
Iteration 2 (learning rate 2, iteration 60), cost: 5.742453675713526, time: 0.5532770156860352
	construction: hidden layer dot costs 0.004005908966064453 secs
	construction: output layer dot costs 0.0010006427764892578 secs
	forward prop: costs 0.04009580612182617 secs
	back prop: costs 0.5059905052185059 secs
Iteration 3 (learning rate 2, iteration 60), cost: 3.676355428184077, time: 0.5525381565093994
	

: 

In [1]:

import functools
import numpy as np
import math
import os
import scipy.io as sio
import time

from mpi4py import MPI
import os

# Thiết lập biến môi trường để đảm bảo rẽ nhánh True
os.environ['MNISTNN_GPU'] = 'yes'
os.environ['MNISTNN_PARALLEL'] = 'yes'

if os.getenv('MNISTNN_GPU') == 'yes':
    Gpu_mode = True
else:
    Gpu_mode = False

if os.getenv('MNISTNN_PARALLEL') == 'yes':
    Distributed = True
else:
    Distributed = False

# if Gpu_mode is True:
#     import theano
#     import theano.tensor as T


# Init MPI
comm = MPI.COMM_WORLD

# Structure of the 3-layer neural network.
Input_layer_size = 400
Hidden_layer_size = 25
Output_layer_size = 10

# Matrix product function.  Default is to use CPU mode.
Matrix_dot = np.dot


def convert_memory_ordering_f2c(array):
    if np.isfortran(array) is True:
        return np.ascontiguousarray(array)
    else:
        return array


def load_training_data(training_file='mnistdata.mat'):
    '''Load training data (mnistdata.mat) and return (inputs, labels).

    inputs: numpy array with size (5000, 400).
    labels: numpy array with size (5000, 1).

    The training data is from Andrew Ng's exercise of the Coursera
    machine learning course (ex4data1.mat).
    '''
  
    training_data = sio.loadmat(training_file)
    inputs = training_data['X'].astype('f8')
    inputs = convert_memory_ordering_f2c(inputs)
    labels = training_data['y'].reshape(training_data['y'].shape[0])
    labels = convert_memory_ordering_f2c(labels)
    return (inputs, labels)


def load_weights(weight_file='mnistweights.mat'):
    '''Load training data (mnistdata.mat) and return (inputs, labels).

    The weights file is from Andrew Ng's exercise of the Coursera
    machine learning course (ex4weights.mat).
    '''
    weights = sio.loadmat(weight_file)
    theta1 = convert_memory_ordering_f2c(weights['Theta1'].astype('f8'))  # size: 25 entries, each has 401 numbers
    theta2 = convert_memory_ordering_f2c(weights['Theta2'].astype('f8'))  # size: 10 entries, each has  26 numbers
    return (theta1, theta2)


def rand_init_weights(size_in, size_out):
    epsilon_init = 0.12
    return np.random.rand(size_out, 1 + size_in) * 2 * epsilon_init - epsilon_init


def sigmoid(z):
    return 1.0 / (1 + pow(math.e, -z))


def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))


if Gpu_mode is True:
    def gpu_matrix_dot():
        time_start = time.time()
    
        x = np.random.rand(1000, 1000).astype(np.float32)
        y = np.random.rand(1000, 1000).astype(np.float32)
        z = np.dot(x, y)  # Nhân ma trận trong NumPy
        
        def f(x, y):
            return np.dot(x, y)  # Hàm thực hiện nhân ma trận
        
        time_end = time.time()
        print('NumPy function creation costs {} secs'.format(time_end - time_start))
        return f
else:
    def gpu_matrix_dot():
        pass


def cost_function(theta1, theta2, input_layer_size, hidden_layer_size, output_layer_size, inputs, labels, regular=0):
    '''
    Note: theta1, theta2, inputs, labels are numpy arrays:

        theta1: (25, 401)
        theta2: (10, 26)
        inputs: (5000, 400)
        labels: (5000, 1)
    '''
    input_layer = np.insert(inputs, 0, 1, axis=1)  # add bias, 5000x401

    time_start = time.time()
    hidden_layer = Matrix_dot(input_layer, theta1.T)
    hidden_layer = sigmoid(hidden_layer)
    hidden_layer = np.insert(hidden_layer, 0, 1, axis=1)  # add bias, 5000x26
    time_end = time.time()
    if comm.rank == 0:
        print('\tconstruction: hidden layer dot costs {} secs'.format(time_end - time_start))

    time_start = time.time()
    output_layer = Matrix_dot(hidden_layer, theta2.T)  # 5000x10
    output_layer = sigmoid(output_layer)
    time_end = time.time()
    if comm.rank == 0:
        print('\tconstruction: output layer dot costs {} secs'.format(time_end - time_start))

    # forward propagation: calculate cost
    time_start = time.time()
    cost = 0.0
    for training_index in range(len(inputs)):
        # transform label y[i] from a number to a vector.
        #
        # Note:
        #   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #    1  2  3  4  5  6  7  8  9 10
        #
        #   if y[i] is 0 -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
        #   if y[i] is 1 -> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        outputs = [0] * output_layer_size
        outputs[labels[training_index]-1] = 1

        for k in range(output_layer_size):
            error = -outputs[k] * math.log(output_layer[training_index][k]) - (1 - outputs[k]) * math.log(1 - output_layer[training_index][k])
            cost += error
    cost /= len(inputs)
    time_end = time.time()
    if comm.rank == 0:
        print('\tforward prop: costs {} secs'.format(time_end - time_start))

    # back propagation: calculate gradiants
    time_start = time.time()
    theta1_grad = np.zeros_like(theta1)  # 25x401
    theta2_grad = np.zeros_like(theta2)  # 10x26
    for index in range(len(inputs)):
        # transform label y[i] from a number to a vector.
        outputs = np.zeros((1, output_layer_size))  # (1,10)
        outputs[0][labels[index]-1] = 1

        # calculate delta3
        delta3 = (output_layer[index] - outputs).T  # (10,1)

        # calculate delta2
        z2 = Matrix_dot(theta1, input_layer[index:index+1].T)  # (25,401) x (401,1)
        z2 = np.insert(z2, 0, 1, axis=0)  # add bias, (26,1)
        delta2 = np.multiply(
            Matrix_dot(theta2.T, delta3),  # (26,10) x (10,1)
            sigmoid_gradient(z2)  # (26,1)
        )
        delta2 = delta2[1:]  # (25,1)

        # calculate gradients of theta1 and theta2
        # (25,401) = (25,1) x (1,401)
        theta1_grad += Matrix_dot(delta2, input_layer[index:index+1])
        # (10,26) = (10,1) x (1,26)
        theta2_grad += Matrix_dot(delta3, hidden_layer[index:index+1])
    theta1_grad /= len(inputs)
    theta2_grad /= len(inputs)
    time_end = time.time()
    if comm.rank == 0:
        print('\tback prop: costs {} secs'.format(time_end - time_start))

    return cost, (theta1_grad, theta2_grad)


def gradient_descent(inputs, labels, learningrate=0.8, iteration=50):
    '''
    @return cost and trained model (weights).
    '''
    if Distributed is True:
        if comm.rank == 0:
            theta1 = rand_init_weights(Input_layer_size, Hidden_layer_size)
            theta2 = rand_init_weights(Hidden_layer_size, Output_layer_size)
        else:
            theta1 = np.zeros((Hidden_layer_size, Input_layer_size + 1))
            theta2 = np.zeros((Output_layer_size, Hidden_layer_size + 1))
        comm.Barrier()
        if comm.rank == 0:
            time_bcast_start = time.time()
        comm.Bcast([theta1, MPI.DOUBLE])
        comm.Barrier()
        comm.Bcast([theta2, MPI.DOUBLE])
        if comm.rank == 0:
            time_bcast_end = time.time()
            print('\tBcast theta1 and theta2 uses {} secs.'.format(time_bcast_end - time_bcast_start))
    else:
        theta1 = rand_init_weights(Input_layer_size, Hidden_layer_size)
        theta2 = rand_init_weights(Hidden_layer_size, Output_layer_size)

    cost = 0.0
    for i in range(iteration):
        time_iter_start = time.time()

        if Distributed is True:
            # Scatter training data and labels.
            
            sliced_inputs = np.asarray(np.split(inputs, comm.size))
            sliced_labels = np.asarray(np.split(labels, comm.size))
            inputs_buf = np.zeros((int(len(inputs)/comm.size), Input_layer_size))
            labels_buf = np.zeros(int(len(labels)/comm.size), dtype='uint8')

            comm.Barrier()
            if comm.rank == 0:
                time_scatter_start = time.time()
            comm.Scatter(sliced_inputs, inputs_buf)
            if comm.rank == 0:
                time_scatter_end = time.time()
                print('\tScatter inputs uses {} secs.'.format(time_scatter_end - time_scatter_start))

            comm.Barrier()
            if comm.rank == 0:
                time_scatter_start = time.time()
            comm.Scatter(sliced_labels, labels_buf)
            if comm.rank == 0:
                time_scatter_end = time.time()
                print('\tScatter labels uses {} secs.'.format(time_scatter_end - time_scatter_start))

            # Calculate distributed costs and gradients of this iteration
            # by cost function.
            comm.Barrier()
            cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2,
                Input_layer_size, Hidden_layer_size, Output_layer_size,
                inputs_buf, labels_buf, regular=0)

            # Gather distributed costs and gradients.
            comm.Barrier()
            cost_buf = [0] * comm.size
            try:
                cost_buf = comm.gather(cost)
                cost = sum(cost_buf) / len(cost_buf)
            except TypeError as e:
                print('[{0}] {1}'.format(comm.rank, e))

            theta1_grad_buf = np.asarray([np.zeros_like(theta1_grad)] * comm.size)
            comm.Barrier()
            if comm.rank == 0:
                time_gather_start = time.time()
            comm.Gather(theta1_grad, theta1_grad_buf)
            if comm.rank == 0:
                time_gather_end = time.time()
                print('\tGather theta1 uses {} secs.'.format(time_gather_end - time_gather_start))
            comm.Barrier()
            theta1_grad = functools.reduce(np.add, theta1_grad_buf) / comm.size

            theta2_grad_buf = np.asarray([np.zeros_like(theta2_grad)] * comm.size)
            comm.Barrier()
            if comm.rank == 0:
                time_gather_start = time.time()
            comm.Gather(theta2_grad, theta2_grad_buf)
            if comm.rank == 0:
                time_gather_end = time.time()
                print('\tGather theta2 uses {} secs.'.format(time_gather_end - time_gather_start))
            comm.Barrier()
            theta2_grad = functools.reduce(np.add, theta2_grad_buf) / comm.size
        else:
            cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2,
                Input_layer_size, Hidden_layer_size, Output_layer_size,
                inputs, labels, regular=0)

        theta1 -= learningrate * theta1_grad
        theta2 -= learningrate * theta2_grad

        if Distributed is True:
           # Sync-up weights for distributed worknodes.
           comm.Bcast([theta1, MPI.DOUBLE])
           comm.Bcast([theta2, MPI.DOUBLE])
           comm.Barrier()

        time_iter_end = time.time()
        if comm.rank == 0:
            print('Iteration {0} (learning rate {2}, iteration {3}), cost: {1}, time: {4}'.format(
                i+1, cost, learningrate, iteration, time_iter_end - time_iter_start)
            )
    return cost, (theta1, theta2)


def train(inputs, labels, learningrate=0.8, iteration=50):
    cost, model = gradient_descent(inputs, labels, learningrate, iteration)
    return model


def predict(model, inputs):
    theta1, theta2 = model
    a1 = np.insert(inputs, 0, 1, axis=1)  # add bias, (5000,401)
    a2 = np.dot(a1, theta1.T)  # (5000,401) x (401,25)
    a2 = sigmoid(a2)
    a2 = np.insert(a2, 0, 1, axis=1)  # add bias, (5000,26)
    a3 = np.dot(a2, theta2.T)  # (5000,26) x (26,10)
    a3 = sigmoid(a3)  # (5000,10)
    return [i.argmax()+1 for i in a3]


if __name__ == '__main__':
    if Gpu_mode is True:
        print('GPU mode')
        Matrix_dot = gpu_matrix_dot()
    else:
        print('CPU mode')
        Matrix_dot = np.dot

    if Distributed is True:
        print('Parallelism: yes')
    else:
        print('Parallelism: no')

    # Note: There are 10 units which present the digits [1-9, 0]
    # (in order) in the output layer.
    inputs, labels = load_training_data()

    # train the model from scratch and predict based on it
    model = train(inputs, labels, learningrate=2, iteration=60)

    # Load pretrained weights for debugging precision.
    # The precision will be around 97% (0.9756).
    #weights = load_weights()
    #theta1 = weights[0]  # size: 25 entries, each has 401 numbers
    #theta2 = weights[1]  # size: 10 entries, each has  26 numbers
    #model = (theta1, theta2)
    #cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2, Input_layer_size, Hidden_layer_size, Output_layer_size, inputs, labels, regular=0)
    #print('cost:', cost)

    outputs = predict(model, inputs)

    correct_prediction = 0
    for i, predict in enumerate(outputs):
        if predict == labels[i]:
            correct_prediction += 1
    precision = float(correct_prediction) / len(labels)
    print('precision: {}'.format(precision))


GPU mode
NumPy function creation costs 0.03338813781738281 secs
Parallelism: yes
	Bcast theta1 and theta2 uses 0.0014274120330810547 secs.
1
	Scatter inputs uses 0.006052732467651367 secs.
	Scatter labels uses 0.0 secs.
	construction: hidden layer dot costs 0.0099029541015625 secs
	construction: output layer dot costs 0.0020132064819335938 secs
	forward prop: costs 0.07876014709472656 secs
	back prop: costs 0.8240408897399902 secs
	Gather theta1 uses 0.0 secs.
	Gather theta2 uses 0.0 secs.
Iteration 1 (learning rate 2, iteration 60), cost: 6.992027750572543, time: 0.9359138011932373
1
	Scatter inputs uses 0.004002809524536133 secs.
	Scatter labels uses 0.0 secs.
	construction: hidden layer dot costs 0.011059999465942383 secs
	construction: output layer dot costs 0.0019986629486083984 secs
	forward prop: costs 0.07767319679260254 secs
	back prop: costs 0.6969900131225586 secs
	Gather theta1 uses 0.0 secs.
	Gather theta2 uses 0.0 secs.
Iteration 2 (learning rate 2, iteration 60), cost: 5

In [ ]:
# mpiexec -np 4 python mnist-nn-data-parallelism.py

In [1]:
import cupy as cp  # Thay np bằng cp để chạy trên GPU
import scipy.io as sio
from sklearn.model_selection import train_test_split
import time

# Cấu trúc mạng nơ-ron
Input_layer_size = 400
Hidden_layer_size = 25
Output_layer_size = 10

# Hàm tiện ích
def convert_memory_ordering_f2c(array):
    """Chuyển ma trận sang định dạng C nếu đang ở định dạng Fortran."""
    return cp.ascontiguousarray(array) if cp.isfortran(array) else array

# Vector hóa sigmoid để tăng hiệu suất trên GPU
sigmoid = cp.vectorize(lambda z: 1.0 / (1 + cp.exp(-z)))
sigmoid_gradient = cp.vectorize(lambda z: sigmoid(z) * (1 - sigmoid(z)))

def load_and_split_data(training_file='mnistdata.mat', test_size=0.2, random_state=42):
    """Tải và chia dữ liệu MNIST thành tập train/test, chuyển sang GPU."""
    training_data = sio.loadmat(training_file)
    inputs = convert_memory_ordering_f2c(training_data['X'].astype('float32'))
    labels = convert_memory_ordering_f2c(training_data['y'].ravel())
    X_train, X_test, y_train, y_test = train_test_split(
        inputs, labels, test_size=test_size, random_state=random_state
    )
    # Chuyển dữ liệu sang GPU
    return (cp.array(X_train), cp.array(X_test), 
            cp.array(y_train), cp.array(y_test))

def rand_init_weights(size_in, size_out):
    """Khởi tạo weights ngẫu nhiên bằng phương pháp Xavier trên GPU."""
    epsilon_init = cp.sqrt(6) / cp.sqrt(size_in + size_out)
    return cp.random.randn(size_out, size_in + 1) * epsilon_init

def cost_function(theta1, theta2, inputs, labels, regular=0.01):
    """Tính hàm mất mát và gradient trên GPU."""
    m = len(inputs)
    
    # Forward propagation
    a1 = cp.hstack([cp.ones((m, 1)), inputs])  # (m, 401)
    z2 = cp.dot(a1, theta1.T)                  # (m, 25)
    a2 = sigmoid(z2)
    a2 = cp.hstack([cp.ones((m, 1)), a2])     # (m, 26)
    z3 = cp.dot(a2, theta2.T)                  # (m, 10)
    h = sigmoid(z3)
    
    # Chuyển labels thành ma trận one-hot
    y_matrix = cp.eye(Output_layer_size)[labels-1]
    
    # Tính cost với regularization
    cost = (-1/m) * cp.sum(y_matrix * cp.log(h) + (1 - y_matrix) * cp.log(1 - h))
    reg = (regular/(2*m)) * (cp.sum(cp.square(theta1[:, 1:])) + cp.sum(cp.square(theta2[:, 1:])))
    total_cost = cost + reg
    
    # Backpropagation
    delta3 = h - y_matrix                      # (m, 10)
    delta2 = cp.dot(delta3, theta2[:, 1:]) * sigmoid_gradient(z2)  # (m, 25)
    
    theta2_grad = (1/m) * cp.dot(delta3.T, a2) + (regular/m) * cp.hstack([cp.zeros((Output_layer_size, 1)), theta2[:, 1:]])
    theta1_grad = (1/m) * cp.dot(delta2.T, a1) + (regular/m) * cp.hstack([cp.zeros((Hidden_layer_size, 1)), theta1[:, 1:]])
    
    return total_cost, (theta1_grad, theta2_grad)

def gradient_descent(inputs, labels, learning_rate=0.8, iterations=50):
    """Huấn luyện mạng nơ-ron bằng gradient descent trên GPU."""
    # Khởi tạo weights trên GPU
    theta1 = rand_init_weights(Input_layer_size, Hidden_layer_size)
    theta2 = rand_init_weights(Hidden_layer_size, Output_layer_size)
    
    # Gradient descent
    for i in range(iterations):
        iter_start = time.time()
        cost, (theta1_grad, theta2_grad) = cost_function(theta1, theta2, inputs, labels)
        theta1 -= learning_rate * theta1_grad
        theta2 -= learning_rate * theta2_grad
        iter_time = time.time() - iter_start
        # Chuyển cost về CPU để in
        cost_cpu = cp.asnumpy(cost)
        print(f"Iteration {i+1}/{iterations}, Cost: {cost_cpu:.4f}, Time: {iter_time:.2f}s")
    
    return cost, (theta1, theta2)

def predict(model, inputs):
    """Dự đoán nhãn từ inputs trên GPU."""
    theta1, theta2 = model
    a1 = cp.hstack([cp.ones((len(inputs), 1)), inputs])
    a2 = sigmoid(cp.dot(a1, theta1.T))
    a2 = cp.hstack([cp.ones((len(a2), 1)), a2])
    h = sigmoid(cp.dot(a2, theta2.T))
    return cp.argmax(h, axis=1) + 1

if __name__ == "__main__":
    print("Running in GPU mode with CuPy")
    
    # Load và chia dữ liệu, chuyển sang GPU
    X_train, X_test, y_train, y_test = load_and_split_data()
    
    # Đo thời gian huấn luyện
    start_time = time.time()
    
    # Huấn luyện mô hình trên GPU
    cost, model = gradient_descent(X_train, y_train, learning_rate=2, iterations=60)
    
    # Tính thời gian huấn luyện
    training_time = time.time() - start_time
    
    # Đánh giá mô hình
    train_pred = predict(model, X_train)
    train_accuracy = cp.mean(train_pred == y_train)
    
    test_pred = predict(model, X_test)
    test_accuracy = cp.mean(test_pred == y_test)
    
    # Chuyển kết quả về CPU để in
    train_accuracy_cpu = cp.asnumpy(train_accuracy)
    test_accuracy_cpu = cp.asnumpy(test_accuracy)
    
    # In kết quả
    print(f"\nTraining accuracy: {train_accuracy_cpu*100:.2f}%")
    print(f"Test accuracy: {test_accuracy_cpu*100:.2f}%")
    print(f"Total training time: {training_time:.2f} seconds")

ModuleNotFoundError: No module named 'cupy'